In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import os

# Computation of the statical polarizability 

This nb assume that the GS analysis for a set of molecule has been performed. The structure of this working directory consists in a folder for each molecule. Inside each folder there is one logifle (or more if different study varying for instance xc functional or equilibrium position) that contains the result of the GS computation used as input of the present analysis

Scan the path folder and build the molecules_database by looking at the directories in the path

In [2]:
molecules_database = os.listdir('.')
for f in reversed(molecules_database):
    if os.path.isdir(f) == False:
        molecules_database.remove(f)
if os.path.isdir('.ipynb_checkpoints'):
    molecules_database.remove('.ipynb_checkpoints')
    
molecules_database

['H20', 'CO']

Choose a molecule to perform the analysis and move the path in the associated folder

In [3]:
molecule = molecules_database[1]
%cd $molecule

/home/marco/Data/RICERCA/LINEAR RESPONSE/LR-nb/STATICAL POLARIZABILITY/CO


Get the relevant parameters of the gs analysis

In [4]:
posinp_file = molecule+'_posinp.xyz'
gs = lf.Logfile('log-lda.yaml')

Build the input file and define the parameters common to all the runs.

The values of rmult and of the field intensity will be specified subsequently since a convergence study will be performed on this parameters.

In [5]:
inp = I.Inputfile()
inp.set_hgrid(gs.log['dft']['hgrids'])
inp.set_xc('LDA')
inp.set_wavefunction_convergence(gnrm=1.0e-5)
inp

{'dft': {'gnrm_cv': 1e-05, 'hgrids': 0.37, 'ixc': 'LDA'}}

In [6]:
def build_alpha_dataset(**kwargs):
    """
    Create the dataset and append the runs needed to compute the statical polarizability
    for a specific choice of the input parameters . Set also a postprocessing function 
    to extract the value of alpha.
    
    Args:
        kwargs['intensity'] : the intensity of the field
        kwargs['input']     : the input file
        kwargs['posinp']    : the posinp
        kwargs['ppf']       : the postprocessing function
        kwargs['runner']    : the instance of SystemCalculator
    """
    lbl = 'alpha_'+str(kwargs['intensity'])
    study = D.Dataset(label=lbl,run_dir='alpha',intensity=kwargs['intensity'],posinp=kwargs['posinp'])
    study.set_postprocessing_function(kwargs['ppf'])
    
    f = kwargs['intensity']
    inp = kwargs['input']
    for ind,sign in enumerate(['+','-']):
        for idir,coord in enumerate(['x','y','z']):
            el=np.zeros(3)
            el[idir]=(1-2*ind)*f
            inp.apply_electric_field(el.tolist())
            idd = {'rmult':inp['dft']['rmult'][0],'dir':coord,'sign':sign,'F':f}
            study.append_run(id=idd,runner=kwargs['runner'],input=inp)
    
    return study

def eval_alpha(study):
    """"
    Extract the statical polarizability tensor from the study dataset
    """
    dipoles = study.fetch_results(attribute = 'dipole')
    f = study.get_global_option('intensity')
    alpha=np.mat(np.zeros(9)).reshape(3,3)
    for ind in range(3):
        alpha[ind] = np.array(dipoles[ind])-np.array(dipoles[ind+3])
    alpha = alpha.T / (2.0*f)
    return alpha

In [7]:
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


Example of usage of this functionality

In [8]:
inp.set_rmult(gs.log['dft']['rmult'])
f = 1e-2

In [9]:
a = build_alpha_dataset(intensity=f,input=inp,runner=code,posinp=posinp_file,ppf=eval_alpha)
write(a.global_options())
ind = 0
write(a.runs[ind])

{'run_dir': 'alpha', 'intensity': 0.01, 'posinp': 'CO_posinp.xyz', 'label': 'alpha_0.01'}
{'run_dir': 'alpha', 'input': {'dft': {'ixc': 'LDA', 'gnrm_cv': 1e-05, 'elecfield': [0.01, 0.0, 0.0], 'hgrids': 0.37, 'rmult': [7.0, 9.0]}}, 'intensity': 0.01, 'posinp': 'CO_posinp.xyz', 'label': 'alpha_0.01'}


In [10]:
alpha = a.run()
alpha

Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:x,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:y,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:y,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:z,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:z,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:-.yaml"
Run directory alpha
Executing co

matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
        [-1.279955e-05,  1.323614e+01, -3.389697e-05],
        [-1.125000e-04, -1.125000e-04,  1.856809e+01]])

Convergence study w.r.t. the intensity of the field.

Perform a comparison among the alpha computed with different field intensity of 1e-2 and 5e-3 and perform a further computation with a field equal to 1e-3 if the alphas differ more than a given tolerance. The comparison is performed with the allcloe functions that returns true if

abs(a-b) < atol + rtol*abs(b)

So for small elements that are deeply affected by numerical noise the maximum allowed discrepacy is atol while for the relevan ones the percentage difference is rtol. 

In [11]:
field_int = [1e-2,5e-3,1e-3]
at = 1e-3
rt = 1e-3

In [12]:
alpha_rmult = {}

In [13]:
alphas = {}

for f in field_int[:2]:
    a = build_alpha_dataset(intensity=f,input=inp,runner=code,posinp=posinp_file,ppf=eval_alpha)
    alphas[f] = a.run()

convergence = np.allclose(alphas[field_int[0]],alphas[field_int[1]],atol = at, rtol = rt)
if convergence:
    write('')
    write('Convergence achieved for field intensity ',field_int[0])
    alpha_rmult[inp['dft']['rmult'][0]] = alphas[field_int[0]]
else:
    write('')
    write('Convergence for field intensity ',field_int[0],' failed')
    write('Reduce the value of the field')
    a = build_alpha_dataset(intensity=field_int[2],input=inp,runner=code,posinp=posinp_file,ppf=eval_alpha)
    alphas[field_int[2]] = a.run()
    convergence = np.allclose(alphas[field_int[1]],alphas[field_int[2]],atol = at, rtol = rt)
    if convergence:
        write('')
        write('Convergence achieved for field intensity ',field_int[1])
        alpha_rmult[inp['dft']['rmult'][0]] = alphas[field_int[1]]
    else:
        write('')
        write('Convergence for field intensity ',field_int[1],' failed')
        write('Return the value of alpha associated to ',field_int[1],'. Perform further check!!!')
        alpha_rmult[inp['dft']['rmult'][0]] = alphas[field_int[1]]


Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:x,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:y,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:y,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:z,rmult:7.0,sign:+.yaml"
Run directory alpha
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n F:0.01,dir:z,rmult:7.0,sign:+ -s Yes
Copy the posinp file 'CO_posinp.xyz' into 'alpha'
Creating the yaml input file "alpha/F:0.01,dir:x,rmult:7.0,sign:-.yaml"
Run directory alpha
Executing co

In [14]:
alpha_rmult

{7.0: matrix([[ 1.3211721e+01, -4.5186718e-04, -2.4358770e-05],
         [-4.5186719e-04,  1.3211721e+01, -2.4358770e-05],
         [ 1.0600000e-04,  1.0600000e-04,  1.8530906e+01]])}

In [ ]:
####################################################################################################

Perform a comparison among the alpha computed with different field and perform a further computation with a field
equal to 1e-3 if the alphas differ more than a given tolerance. The comparison is performed with the allcloe functions that returns true if

abs(a-b) < atol + rtol*abs(b)

So for small elements that are deeply affected by numerical noise the maximum allowed discrepacy is atol while for the relevan ones the percentage difference is rtol. 

In [9]:
at = 1e-3
rt = 1e-2

if the comparison returns False we need to perform a further computation for the lowest value of the field intensity

In [10]:
convergence = np.allclose(alphas[field_intensity[0]],alphas[field_intensity[1]],atol = at, rtol = rt)
if convergence:
    write('Convergence achieved for field intensity ',field_intensity[0])
    alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[0]]
else:
    write('Convergence for field intensity ',field_intensity[0],' failed')
    write('Reduce the value of the field')
    append_electric_field_runs(study,field_intensity[2],inp)
    study.run()
    dipoles = study.fetch_results({'F':field_intensity[2]},attribute = 'dipole')
    alphas[field_intensity[2]] = eval_alpha_one_field(dipoles,field_intensity[2])
    convergence = np.allclose(alphas[field_intensity[0]],alphas[field_intensity[1]],atol = at, rtol = rt)
    if convergence:
        write('Convergence achieved for field intensity ',field_intensity[1])
        alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[1]]
    else:
        write('Convergence for field intensity ',field_intensity[1],' failed')
        write('Return the value of alpha associated to ',field_intensity[1],'. Perform further check!!!')
        alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[1]]

NameError: name 'alphas' is not defined

In [ ]:
#################################################

Define some useful functions

In [16]:

def append_electric_field_runs(study,intensity,inp):
    """
    Append 6 runs (one for each direction of the field including both positive and negative values) to
    the dataset
    
    Args:
        study (dataset): the dataset
        intensity (float): the field intensity
        inp (inputFile): the input file of the run 
    """
    for ind,sign in enumerate(['+','-']):
        for idir,coord in enumerate(['x','y','z']):
            el=np.zeros(3)
            el[idir]=(1-2*ind)*intensity
            inp.apply_electric_field(el.tolist())
            study.append_run({'rmult':inp['dft']['rmult'][0],'dir':coord,'sign':sign,'F':intensity},code,input=inp)

Define global variables

In [57]:
def eval_alpha_one_field(dipoles,f):
    """"
    Compute alpha given one value of the field intensity and the associated list of 
    six dipoles
    """
    alpha=np.mat(np.zeros(9)).reshape(3,3)
    for ind in range(3):
        alpha[ind] = np.array(dipoles[ind])-np.array(dipoles[ind+3])
    alpha = alpha.T / (2.0*f)
    return alpha

Introduce the alphas dictionary that contains the polarizability tensors for each value of the field intensity. Needed for the convergence study

In [58]:
alphas = {}

In [59]:
for f in field_intensity[:2]:
    dipoles = study.fetch_results({'F':f},attribute = 'dipole')
    alphas[f] = eval_alpha_one_field(dipoles,f)
alphas

{0.005: matrix([[ 1.3211721e+01, -4.5186718e-04, -2.4358770e-05],
         [-4.5186719e-04,  1.3211721e+01, -2.4358770e-05],
         [ 1.0600000e-04,  1.0600000e-04,  1.8530906e+01]]),
 0.01: matrix([[ 1.323614e+01, -1.279955e-05, -3.389697e-05],
         [-1.279955e-05,  1.323614e+01, -3.389697e-05],
         [-1.125000e-04, -1.125000e-04,  1.856809e+01]])}

Perform a comparison among the alpha computed with different field and perform a further computation with a field
equal to 1e-3 if the alphas differ more than a given tolerance. The comparison is performed with the allcloe functions that returns true if

abs(a-b) < atol + rtol*abs(b)

So for small elements that are deeply affected by numerical noise the maximum allowed discrepacy is atol while for the relevan ones the percentage difference is rtol. 

In [9]:
at = 1e-3
rt = 1e-2

if the comparison returns False we need to perform a further computation for the lowest value of the field intensity

In [10]:
convergence = np.allclose(alphas[field_intensity[0]],alphas[field_intensity[1]],atol = at, rtol = rt)
if convergence:
    write('Convergence achieved for field intensity ',field_intensity[0])
    alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[0]]
else:
    write('Convergence for field intensity ',field_intensity[0],' failed')
    write('Reduce the value of the field')
    append_electric_field_runs(study,field_intensity[2],inp)
    study.run()
    dipoles = study.fetch_results({'F':field_intensity[2]},attribute = 'dipole')
    alphas[field_intensity[2]] = eval_alpha_one_field(dipoles,field_intensity[2])
    convergence = np.allclose(alphas[field_intensity[0]],alphas[field_intensity[1]],atol = at, rtol = rt)
    if convergence:
        write('Convergence achieved for field intensity ',field_intensity[1])
        alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[1]]
    else:
        write('Convergence for field intensity ',field_intensity[1],' failed')
        write('Return the value of alpha associated to ',field_intensity[1],'. Perform further check!!!')
        alpha[inp['dft']['rmult'][0]] = alphas[field_intensity[1]]

NameError: name 'alphas' is not defined

In [ ]:
#############################################################

The aim of this nb is to provide all the tools needed to compute the statical polarizability of a molecule.
The results of this analysis allow us to perform convergence study of the statical polarizability w.r.t the size of the simulation domain and to check if the strenght of the field used to perturb the ground state configuration of the molecule is compatible with the linear response regime 

The data structure which encodes all the computation that will be performed during the analysis is a python dictionary generically called molecule with structure:
* molecule[rmult][field]

where [field] is a tuple with the cartesian components of the statical field

In order to perform this analysis several methods need to be defined, in particular:
 * computeStatPol(molecule,fieldNorm,box) : take as inputs the properties of the choses molecule, the value of the field norm and the value of the box size (rmult in our specific case). This method check if the results associated to the given field norm and box value are present in molecule. If needed perform the computation storing the results in molecule and add the 'statPol' key to molecule with the (matrix valued) result for the statical polarizability

Let us start this analysis with some specific examples